In [1]:
import advectParticles

In [2]:
advectParticles.gridAdvection()

int

In [3]:
class myGrid:
    def __init__(self, nlon, nlat, minLat=60., maxLat=90., minLon=-180, maxLon=180):
        self.nlat = nlat
        self.nlon = nlon
        self.dlat = (maxLat - minLat)/nlat
        self.dlon = 360/nlon

In [4]:
foo = myGrid(10,20)

In [5]:
type(foo)

__main__.myGrid

In [3]:
import xarray as xr

In [4]:
ds = xr.open_dataset('/data/oceanparcels/output_data/data_Daan/pset_y300_P3590x590_D90_DT60_ODT12_LAT60.5-89.5_LON-179.5-179.5.nc')

In [ ]:
ds.time

In [3]:
k = str(103919595)

In [4]:
k[0:4]

'1039'